In [3]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential, Model
from keras.layers import Input, Dense
from keras.utils import to_categorical 

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

df = pd.read_csv('datamahasiswa2015.csv')
df.head()

Using TensorFlow backend.


,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,Desjar,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,3.5,2.0,TECHNO,TECHNO,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,3.5,ERP,ERP,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,OUT,OUT,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,4.0,EAD,EAD,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,2.0,1.0,OUT,OUT,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU


In [4]:
new_class = [0] * len(df['Keterangan'])
for index, row in df.iterrows() :
    if row['Keterangan'] == "TEPAT WAKTU" :
        new_class[index] = 1
    else :
        new_class[index] = 0

In [5]:
df["ket_status"] = new_class
df.head()

,No,NIM,OOP,RPB,RPL,PI,APSI,WEB,Statistik,Matdis,...,KSI,Pilihan 1,Peminatan,MP1,MP2,MP3,Status,Tanggal Lulus,Keterangan,ket_status
0,1,1202150001,2.0,3.5,0.0,3.5,3.0,3.0,2.0,2.0,...,2.0,TECHNO,TECHNO,3.5,3.0,3.0,GRADUATED,30/08/19,TEPAT WAKTU,1
1,2,1202150002,3.0,3.5,3.5,4.0,3.0,3.0,3.5,3.5,...,3.5,ERP,ERP,4.0,4.0,3.5,GRADUATED,10/07/19,TEPAT WAKTU,1
2,3,1202150003,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,OUT,OUT,0.0,0.0,0.0,RESIGN,OUT,TIDAK TEPAT WAKTU,0
3,4,1202150004,3.5,4.0,3.5,4.0,3.5,4.0,3.5,3.5,...,4.0,EAD,EAD,3.5,4.0,2.0,GRADUATED,05/07/19,TEPAT WAKTU,1
4,5,1202150005,2.0,2.5,2.0,1.0,3.5,1.0,2.0,1.0,...,1.0,OUT,OUT,0.0,0.0,0.0,STUDENT,OUT,TIDAK TEPAT WAKTU,0


In [72]:
mainData = df.drop(['Status','Keterangan','Pilihan 1','Peminatan','Tanggal Lulus','No','NIM','MP1','MP2','MP3','RPL'],axis=1)
mainData.rename(columns={'KSI':'Manprosi'}, inplace=True)
mainData.head()

,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,ket_status
0,2.0,3.5,3.5,3.0,3.0,2.0,2.0,3.0,2.0,3.5,3.5,3.0,3.5,2.0,2.0,2.5,3.5,3.5,2.0,1
1,3.0,3.5,4.0,3.0,3.0,3.5,3.5,3.5,3.0,3.5,4.0,4.0,3.0,3.5,2.5,3.5,3.5,3.5,3.5,1
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
3,3.5,4.0,4.0,3.5,4.0,3.5,3.5,3.5,4.0,3.0,4.0,4.0,3.0,3.5,3.5,3.5,3.5,4.0,4.0,1
4,2.0,2.5,1.0,3.5,1.0,2.0,1.0,2.5,0.0,2.5,4.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,0


In [73]:
target = ["ket_status"]
predictors = list(set(list(mainData.columns))-set(target))
mainData[predictors] = mainData[predictors]/mainData[predictors].max()
mainData[predictors].head()

,SCM,Sisop,Strukdat,Desjar,Matdis,Basdat,APSI,RPB,WEB,Manjarkom,SE,EA,PO,OOP,MSDM,Alpro,Statistik,Manprosi,PI
0,0.75,0.625,0.50,0.875,0.500,0.500,0.750,0.875,0.75,0.500,0.875,0.875,0.875,0.500,0.875,0.750,0.500,0.500,0.875
1,1.00,0.875,0.75,0.875,0.875,0.875,0.750,0.875,0.75,0.625,0.875,0.750,1.000,0.750,0.875,0.875,0.875,0.875,1.000
2,0.00,0.000,0.50,0.250,0.000,0.000,0.250,0.000,0.00,0.000,0.500,0.000,0.000,0.250,0.000,0.000,0.000,0.000,0.000
3,1.00,0.875,1.00,1.000,0.875,0.875,0.875,1.000,1.00,0.875,0.750,0.750,1.000,0.875,0.875,0.875,0.875,1.000,1.000
4,0.50,0.500,0.00,0.500,0.250,0.250,0.875,0.625,0.25,0.500,0.625,0.500,1.000,0.500,0.250,0.625,0.500,0.250,0.250


In [74]:
X = mainData[target]
y = mainData[predictors]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=40)
print(X_train.shape)
print(y_train.shape)
print(X_test.head())
print(y_test.shape)

(259, 1)
(259, 19)
     ket_status
263           1
314           1
296           0
26            1
185           1
(111, 19)


In [75]:
# y_train = to_categorical(y_train)
# y_test = to_categorical(y_test)

X_trains = to_categorical(X_train)
X_tests = to_categorical(X_test)

print(X_trains.shape)

(259, 2)


In [76]:
model = Sequential()
model.add(Dense(500, activation='relu', input_dim=19))
model.add(Dense(300, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [77]:
model.fit(y_train, X_trains, epochs=200)

Epoch 1/200
259/259 [==============================] - 0s 370us/step - loss: 0.5119 - accuracy: 0.7529
Epoch 2/200
259/259 [==============================] - 0s 108us/step - loss: 0.4171 - accuracy: 0.8185
Epoch 3/200
259/259 [==============================] - 0s 108us/step - loss: 0.3447 - accuracy: 0.8533
Epoch 4/200
259/259 [==============================] - 0s 123us/step - loss: 0.3152 - accuracy: 0.8919
Epoch 5/200
259/259 [==============================] - 0s 108us/step - loss: 0.3225 - accuracy: 0.8726
Epoch 6/200
259/259 [==============================] - 0s 123us/step - loss: 0.2850 - accuracy: 0.8726
Epoch 7/200
259/259 [==============================] - 0s 123us/step - loss: 0.2544 - accuracy: 0.9073
Epoch 8/200
259/259 [==============================] - 0s 124us/step - loss: 0.2553 - accuracy: 0.9189
Epoch 9/200
259/259 [==============================] - 0s 108us/step - loss: 0.2845 - accuracy: 0.8726
Epoch 10/200
259/259 [==============================] - 0s 123us/step - l

In [78]:
pred_train= model.predict(y_train)
scores = model.evaluate(y_train, X_trains, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))   
 
pred_test= model.predict(y_test)
scores2 = model.evaluate(y_test, X_tests, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

Accuracy on training data: 0.9845559597015381% 
 Error on training data: 0.015444040298461914
Accuracy on test data: 0.837837815284729% 
 Error on test data: 0.162162184715271


In [79]:
hasil = pd.DataFrame()
hasil["NIM"] = df["NIM"]
hasil["ket_status"] = df["ket_status"]
temp = model.predict_classes(y_test)

# hasil = model.predict_classes(y_test)

In [82]:
pre = [0] * len(hasil)
for index, row in hasil.iterrows():
    if (index in y_test.index) :
        pre[index] = 1
    else :
        pre[index] = "train data"
        
hasil["forecast"] = pre
hasil.head()


,NIM,ket_status,forecast
0,1202150001,1,1
1,1202150002,1,train data
2,1202150003,0,train data
3,1202150004,1,1
4,1202150005,0,1


In [83]:
dataTesting = pd.read_excel('Nilai Master Pipe.xlsx')
dataTesting.head()

,nim,nama,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,...,SE,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi
0,1202170006,SHINTA EKA RISA SINURAYA,4.0,3.5,2.0,2.0,3.5,4.0,2.5,3.5,...,4.0,3.0,3.0,3.0,3.0,3.0,3.0,3.5,3.5,2.5
1,1202170010,COKORDA AGUNG JAYA LAKSMANA PRAKASA,4.0,3.5,4.0,2.5,4.0,3.0,4.0,3.0,...,4.0,3.5,3.0,3.5,3.5,3.5,3.0,3.5,3.5,3.0
2,1202170011,RAFIF FAKHRI RAMADHAN,4.0,3.0,3.5,2.0,3.5,3.5,3.5,3.5,...,4.0,3.5,3.5,3.5,4.0,3.5,3.0,4.0,3.5,3.0
3,1202170015,M IKHSAN PRATAMA P,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,...,3.5,2.5,3.5,3.0,3.5,3.0,3.5,3.5,3.5,2.5
4,1202170018,DINDA SONIA TIARA,2.0,2.5,3.0,3.5,3.5,2.5,2.5,3.0,...,3.5,3.5,2.5,3.5,3.0,2.5,3.0,3.5,2.0,3.5


In [95]:
dataTestingFilter = dataTesting[predictors]
dataTestingHasil = model.predict_classes(dataTestingFilter)

dataTesting["Lulus Tepat Waktu"] = dataTestingHasil
dataTesting.rename(columns={'nim':'NIM'}, inplace=True)
dataTesting.drop(columns="nama",axis=1).head()

,NIM,OOP,RPB,PI,APSI,WEB,Statistik,Matdis,Alpro,Strukdat,...,PO,SCM,EA,Basdat,Manjarkom,Sisop,MSDM,Desjar,Manprosi,Lulus Tepat Waktu
0,1202170006,4.0,3.5,2.0,2.0,3.5,4.0,2.5,3.5,2.5,...,3.0,3.0,3.0,3.0,3.0,3.0,3.5,3.5,2.5,1
1,1202170010,4.0,3.5,4.0,2.5,4.0,3.0,4.0,3.0,4.0,...,3.5,3.0,3.5,3.5,3.5,3.0,3.5,3.5,3.0,1
2,1202170011,4.0,3.0,3.5,2.0,3.5,3.5,3.5,3.5,4.0,...,3.5,3.5,3.5,4.0,3.5,3.0,4.0,3.5,3.0,1
3,1202170015,4.0,3.5,3.5,3.5,3.5,3.5,3.5,4.0,4.0,...,2.5,3.5,3.0,3.5,3.0,3.5,3.5,3.5,2.5,1
4,1202170018,2.0,2.5,3.0,3.5,3.5,2.5,2.5,3.0,3.5,...,3.5,2.5,3.5,3.0,2.5,3.0,3.5,2.0,3.5,1


In [97]:
pred_test= model.predict(dataTestingFilter)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))    

Accuracy on test data: 0.837837815284729% 
 Error on test data: 0.162162184715271
